In [1]:
%matplotlib notebook
#import matplotlib.pyplot as plt
# Note! ITK interacts weirdly here.  from lazy_imports import itk does not work.
# Additionally, import itk must occur before lazy_imports for itkwidgets.view (ie itkview) to work.
import itk
#from lazy_imports import itk
#from lazy_imports import torch
import torch # to avoid sym3eig_cpu ImportError: libc10.so: cannot open shared object file: No such file or directory
from lazy_imports import np
from lazy_imports import plt
from lazy_imports import sitk
from lazy_imports import loadmat, savemat
from lazy_imports import sitk
from lazy_imports import itkwidgets
from lazy_imports import itkview
from lazy_imports import interactive
from lazy_imports import ipywidgets
from lazy_imports import pv
from lazy_imports import vlinalg
from torch_sym3eig import Sym3Eig
plt.rcParams["figure.figsize"] = (6, 6) # (w, h)

In [2]:
from util.SplitEbinMetric3D import get_karcher_mean, Squared_distance_Ebin_field, logm_invB_A

In [3]:
from util.diffeo import get_idty_3d, get_gradient_3d, compose_function_3d

In [4]:
from algo.metricMatching import metric_matching

In [5]:
from disp.vis import show_2d, show_2d_tensors
from disp.vis import vis_tensors, vis_path
from disp.vis import view_3d_tensors, tensors_to_mesh, view_3d_paths, path_to_tube
from data.io import readRaw, ReadScalars, ReadTensors, WriteTensorNPArray, WriteScalarNPArray, readPath3D
from data.convert import GetNPArrayFromSITK, GetSITKImageFromNP

In [6]:
cases = []
#all_start_coords = []
cases.append('103818')
#all_start_coords.append([[62,126,56]])
cases.append('105923')
#all_start_coords.append([[61,125,56]])
cases.append('111312')
#all_start_coords.append([[62,128,56]])
masks = []
t1s = []
in_tensors = []
out_tensors = []
outdirs = []
for run_case in cases:
  subj = run_case[:6]
  outroot='/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python/simulation_results/'
  indir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/prepped_data/' + subj + '/'
  #bval = 3000
  #outroot='/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python/UKF_experiments/'
  #indir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/prepped_UKF_data_with_grad-dev/' + subj + '/'
  bval = 1000
  outdir = f'{outroot}{run_case}/mineval_0.005_n_3000_s_1.5/'
  outdirs.append(outdir)

  tens_file = f'dti_{bval}_tensor.nhdr'
  in_tens = ReadTensors(indir + tens_file)
  tens_file = 'scaled_tensors.nhdr'
  out_tens = ReadTensors(outdir + tens_file)
  t1_file = 't1_stripped_irescaled.nhdr'
  t1 = ReadScalars(indir + t1_file)
  mask_file = 'filt_mask.nhdr'
  mask = ReadScalars(outdir + mask_file)
  in_tensors.append(in_tens)
  out_tensors.append(out_tens)
  t1s.append(t1)
  masks.append(mask)
  #t1_flip = ReadScalars(indir + 't1_rescaled_flipped.nhdr')[:,::-1,:]


#atlasdir = '/home/sci/hdai/Shared/BrainAtlasMay21/'
#atlas_tens = ReadTensors(atlasdir + 'atlas_tens.nhdr')
#atlas_mask = ReadScalars(atlasdir + 'atlas_mask.nhdr')
atlasdir = '/home/sci/hdai/Shared/Brain3AtlasMay24/'
atlas_tens = ReadTensors(atlasdir + 'atlas_800_tens.nhdr')
atlas_mask = ReadScalars(atlasdir + 'atlas_800_mask.nhdr')

In [7]:
%%time
do_GPU=False
#do_GPU=True
if do_GPU:
  torch.set_default_tensor_type('torch.cuda.FloatTensor')
  torch.cuda.set_device(1)
else:
  torch.set_default_tensor_type('torch.FloatTensor')  
tens_0 = np.zeros((out_tensors[0].shape[0],out_tensors[0].shape[1],out_tensors[0].shape[2],3,3))
tens_1 = np.zeros((out_tensors[1].shape[0],out_tensors[1].shape[1],out_tensors[1].shape[2],3,3))
for xx in range(tens_0.shape[0]):
  for yy in range(tens_0.shape[1]):
    for zz in range(tens_0.shape[2]):
      if masks[0][xx,yy,zz]:
        tens_0[xx,yy,zz,0,0] = out_tensors[0][xx,yy,zz,0]
        tens_0[xx,yy,zz,0,1] = out_tensors[0][xx,yy,zz,1]
        tens_0[xx,yy,zz,1,0] = out_tensors[0][xx,yy,zz,1]
        tens_0[xx,yy,zz,0,2] = out_tensors[0][xx,yy,zz,2]
        tens_0[xx,yy,zz,2,0] = out_tensors[0][xx,yy,zz,2]
        tens_0[xx,yy,zz,1,1] = out_tensors[0][xx,yy,zz,3]
        tens_0[xx,yy,zz,1,2] = out_tensors[0][xx,yy,zz,4]
        tens_0[xx,yy,zz,2,1] = out_tensors[0][xx,yy,zz,4]
        tens_0[xx,yy,zz,2,2] = out_tensors[0][xx,yy,zz,5]
      else:
        tens_0[xx,yy,zz,0,0] = 1
        tens_0[xx,yy,zz,1,1] = 1
        tens_0[xx,yy,zz,2,2] = 1
      if masks[1][xx,yy,zz]:
        tens_1[xx,yy,zz,0,0] = out_tensors[1][xx,yy,zz,0]
        tens_1[xx,yy,zz,0,1] = out_tensors[1][xx,yy,zz,1]
        tens_1[xx,yy,zz,1,0] = out_tensors[1][xx,yy,zz,1]
        tens_1[xx,yy,zz,0,2] = out_tensors[1][xx,yy,zz,2]
        tens_1[xx,yy,zz,2,0] = out_tensors[1][xx,yy,zz,2]
        tens_1[xx,yy,zz,1,1] = out_tensors[1][xx,yy,zz,3]
        tens_1[xx,yy,zz,1,2] = out_tensors[1][xx,yy,zz,4]
        tens_1[xx,yy,zz,2,1] = out_tensors[1][xx,yy,zz,4]
        tens_1[xx,yy,zz,2,2] = out_tensors[1][xx,yy,zz,5]
      else:
        tens_1[xx,yy,zz,0,0] = 1
        tens_1[xx,yy,zz,1,1] = 1
        tens_1[xx,yy,zz,2,2] = 1
        

if do_GPU:
  torch_tens_0 = torch.from_numpy(tens_0).cuda().float()
  torch_tens_1 = torch.from_numpy(tens_1).cuda().float()
  mask_union = torch.from_numpy(masks[0]).cuda().float()
  mask_union += torch.from_numpy(masks[1]).cuda().float()

else:
  torch_tens_0 = torch.from_numpy(tens_0).float()
  torch_tens_1 = torch.from_numpy(tens_1).float()
  mask_union = torch.from_numpy(masks[0]).float()
  mask_union += torch.from_numpy(masks[1]).float()
    

CPU times: user 13 s, sys: 292 ms, total: 13.3 s
Wall time: 12.8 s


In [ ]:
%%time
met_0_1_dist = Squared_distance_Ebin_field(torch.inverse(torch_tens_0), 
                                           torch.inverse(torch_tens_1), 
                                           1./2, 
                                           mask_union).cpu().detach().numpy()
# Torch CPU: CPU Time: 26.7 s, Wall time: 21.5 s
# Torch GPU: CPU time: 6 min 48s, Wall time: 6 min 48s 

In [7]:
%%time
G = torch.stack((torch.inverse(torch_tens_0),
                 torch.inverse(torch_tens_1)))
dim = 3.
kmean = get_karcher_mean(G, 1./dim)
# Torch CPU: CPU Time: 24.7 s, Wall time: 10.8 s 
# Torch GPU: CPU Time:  7min 18s, Wall time:  7min 18s

CPU times: user 7min 18s, sys: 75.5 ms, total: 7min 18s
Wall time: 7min 18s


In [8]:
kmean.device

device(type='cuda', index=1)

In [8]:
itkview(met_0_1_dist)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [8]:
itkview(kmean[:,:,:,0,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [9]:
itkview(G[0][:,:,:,0,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [10]:
itkview(G[1][:,:,:,0,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [9]:
#kmean_GPU=kmean.detach().numpy()
#kmean_CPU=kmean.detach().numpy()

In [10]:
#kmean_diff = kmean_GPU-kmean_CPU
#print(np.max(kmean_diff),np.min(kmean_diff))¯

In [8]:
print(torch_tens_1.dtype,torch.inverse(torch_tens_0).dtype)
# TODO in metric_matching, ensure inputs are torch.float32

torch.float32 torch.float32


In [8]:
%%time
#use_idty=True
use_idty=False
dim, sigma, epsilon, iter_num = 3., 0, 5e-3, 3
height, width, depth = 145,174,145
#with torch.autograd.detect_anomaly():
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  tens_0_def, phi_0, phi_inv_0 = metric_matching(torch.inverse(torch_tens_0), 
                                                 torch.inverse(torch_tens_1), 
                                                 height, width, depth, mask_union, iter_num, epsilon, sigma,dim,use_idty)
print(prof.key_averages().table(sort_by="self_cpu_time_total"))
# Torch CPU 3 iters: CPU time: 7 min 17 s, Wall time: 2 min 7s
# Torch CPU 3 iters: CPU time: 4min 7 s, Wall time: 1 min 39s
# Torch GPU w/ anomaly detection 3 iters: CPU time: 40 min 20s, Wall time: 38 min 7s
# Torch GPU 3 iters: CPU time: 39min 38s, Wall time: 37min 55s



515 elements of gi are not symmetric
tensor([[[ 9.9554,  1.1311,  0.5831],
         [ 1.1311, 10.1719,  1.1953],
         [ 0.5831,  1.1953,  9.2194]],

        [[ 6.0318,  1.1220,  0.5670],
         [ 1.1220,  8.2391,  1.3645],
         [ 0.5670,  1.3645,  6.6151]],

        [[ 9.4020,  1.0885,  0.0864],
         [ 1.0885, 10.2546, -0.4213],
         [ 0.0864, -0.4213, 10.0278]],

        ...,

        [[ 2.5409, -0.5618,  0.5896],
         [-0.5618,  1.2932, -0.2101],
         [ 0.5896, -0.2101,  2.1608]],

        [[ 3.1314,  0.6244, -0.0937],
         [ 0.6244,  4.0015, -0.3831],
         [-0.0937, -0.3831,  2.9300]],

        [[ 7.7619,  1.0622,  1.0209],
         [ 1.0622,  7.9103,  1.2072],
         [ 1.0209,  1.2072,  9.2985]]])
164 elements of gi are not symmetric
tensor([[[ 8.5862,  0.7609,  1.0374],
         [ 0.7609, 10.6777,  2.8820],
         [ 1.0374,  2.8820, 10.8382]],

        [[ 4.8994, -0.9666, -0.8564],
         [-0.9666,  4.6840,  0.9180],
         [-0.8564,  0.91

IndexError: select(): index 3658350 out of range for tensor of size [3658350, 3] at dimension 0

In [12]:
#tens_0_idty = tens_0_def
#phi_0_idty = phi_0
#phi_inv_0_idty = phi_inv_0
#tens_0_phi_inv = tens_0_def
#phi_0_phi_inv = phi_0
#phi_inv_0_phi_inv = phi_inv_0
tens_0_def_GPU = tens_0_def.cpu().detach()
phi_0_GPU = phi_0.cpu().detach()
phi_inv_0_GPU = phi_inv_0.cpu().detach()

In [12]:
tens_diff = tens_0_idty - tens_0_phi_inv
phi_diff = phi_0_idty - phi_0_phi_inv
phi_inv_diff = phi_inv_0_idty - phi_inv_0_phi_inv
print(torch.max(tens_diff),torch.min(tens_diff))
print(torch.max(phi_diff),torch.min(phi_diff))
print(torch.max(phi_inv_diff),torch.min(phi_inv_diff))

tensor(0.) tensor(0.)
tensor(0.) tensor(0.)
tensor(0.) tensor(0.)


In [14]:
tens_diff_GPU = tens_0_def_GPU.cpu() - tens_0_phi_inv
phi_diff_GPU = phi_0_GPU.cpu() - phi_0_phi_inv
phi_inv_diff_GPU = phi_inv_0_GPU.cpu() - phi_inv_0_phi_inv
print(torch.max(tens_diff_GPU),torch.min(tens_diff_GPU))
print(torch.max(phi_diff_GPU),torch.min(phi_diff_GPU))
print(torch.max(phi_inv_diff_GPU),torch.min(phi_inv_diff_GPU))

tensor(0.0001) tensor(-9.1553e-05)
tensor(6.1035e-05) tensor(-6.1035e-05)
tensor(6.1035e-05) tensor(-6.1035e-05)


In [24]:
#del tens_0_def_GPU, phi_0_GPU, phi_inv_0_GPU
del tens_0_def, phi_0, phi_inv_0
torch.cuda.empty_cache()

In [20]:
idty = get_idty_3d(height, width, depth)
itkview((phi_inv_0_idty-idty).cpu().detach().numpy()[2,:,:,:])


Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [21]:
itkview((phi_inv_0_phi_inv-idty).cpu().detach().numpy()[2,:,:,:])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [32]:
def new_get_gradient_3d(F):  # 3D F: size_h x size_w x size_d
    print(F.device)
    F_padded = torch.zeros((F.shape[0] + 2, F.shape[1] + 2, F.shape[2] + 2), device=F.device, dtype=torch.double)
    print(F_padded.device)
    F_padded[1:-1, 1:-1, 1:-1] = F
    F_padded[0, :, :] = F_padded[1, :, :]
    F_padded[-1, :, :] = F_padded[-2, :, :]
    F_padded[:, 0, :] = F_padded[:, 1, :]
    F_padded[:, -1, :] = F_padded[:, -2, :]
    F_padded[:, :, 0] = F_padded[:, :, 1]
    F_padded[:, :, -1] = F_padded[:, :, -2]
#     original
#     F_x = (torch.roll(F_padded, shifts=(0, 0, -1), dims=(0, 1, 2))
#            - torch.roll(F_padded, shifts=(0, 0, 1), dims=(0, 1, 2))) / 2
#     F_y = (torch.roll(F_padded, shifts=(0, -1, 0), dims=(0, 1, 2))
#            - torch.roll(F_padded, shifts=(0, 1, 0), dims=(0, 1, 2))) / 2
#     F_z = (torch.roll(F_padded, shifts=(-1, 0, 0), dims=(0, 1, 2))
#            - torch.roll(F_padded, shifts=(1, 0, 0), dims=(0, 1, 2))) / 2
# 4.3 version
    F_x = (torch.roll(F_padded, shifts=(-1, 0, 0), dims=(0, 1, 2))
           - torch.roll(F_padded, shifts=(1, 0, 0), dims=(0, 1, 2))) / 2
    F_y = (torch.roll(F_padded, shifts=(0, -1, 0), dims=(0, 1, 2))
           - torch.roll(F_padded, shifts=(0, 1, 0), dims=(0, 1, 2))) / 2
    F_z = (torch.roll(F_padded, shifts=(0, 0, -1), dims=(0, 1, 2))
           - torch.roll(F_padded, shifts=(0, 0, 1), dims=(0, 1, 2))) / 2
    print(F_x.device)
    print(F_y.device)
    print(F_z.device)
    retval = torch.stack((F_x[1:-1, 1:-1, 1:-1],
                        F_y[1:-1, 1:-1, 1:-1],
                        F_z[1:-1, 1:-1, 1:-1]))
    print(retval.device)
    print(retval.dtype)
    return retval


In [14]:
dim, sigma, epsilon, iter_num = 3., 0, 5e-3, 3
height, width, depth = 145,174,145

gi = torch.inverse(torch_tens_0)
#print(phi_inv.shape, gi.shape)
print(gi.device)
phi_inv = get_idty_3d(height, width, depth).to(gi.device)
print(phi_inv.device)
g = gi.permute(3, 4, 0, 1, 2)
idty = get_idty_3d(*g.shape[-3:]).to(g.device)
print(idty.device)
eye = torch.eye(3, device=phi_inv.device)
print(eye.device)
ones = torch.ones(*g.shape[-3:], device=g.device)
print(ones.device)
print(phi_inv.shape, idty.shape)
phidiff = phi_inv - idty
print(phidiff.device)
grad0 = get_gradient_3d(phidiff[0])
grad1 = get_gradient_3d(phidiff[1])
grad2 = get_gradient_3d(phidiff[2])
print(grad0.device, grad1.device, grad2.device)
jac = torch.stack((grad0, grad1, grad2))
print(jac.device)
#d_phi = get_jacobian_matrix_3d(phi - idty) + torch.einsum("ij,mno->ijmno", eye, ones)
d_phi = jac + torch.einsum("ij,mno->ijmno", eye, ones)
print(d_phi.device)
g_phi = compose_function_3d(g, phi_inv)
phi_gi = torch.einsum("ij...,ik...,kl...->...jl", d_phi, g_phi, d_phi)


cuda:1
cuda:1
cuda:1
cuda:1
cuda:1
torch.Size([3, 145, 174, 145]) torch.Size([3, 145, 174, 145])
cuda:1
cuda:1 cuda:1 cuda:1
cuda:1
cuda:1


In [15]:
delt = (phi_gi - gi).cpu().detach().numpy()
print(np.max(delt),np.min(delt))

4.765183998500788e-07 -4.7452222595723015e-07


In [12]:
#tens_0_def_CPU = tens_0_def.detach().numpy()
#phi_0_CPU = phi_0.detach().numpy()
#phi_inv_0_CPU = phi_inv_0.detach().numpy()
#tens_0_def_GPU = tens_0_def.detach().numpy()
#phi_0_GPU = phi_0.detach().numpy()
#phi_inv_0_GPU = phi_inv_0.detach().numpy()

In [13]:
#tens_def_diff = tens_0_def_CPU-tens_0_def_GPU
#phi_diff = phi_0_CPU - phi_0_GPU
#phi_inv_diff = phi_inv_0_CPU - phi_inv_0_GPU
#print(np.max(tens_def_diff),np.min(tens_def_diff))
#print(np.max(phi_diff),np.min(phi_diff))
#print(np.max(phi_inv_diff),np.min(phi_inv_diff))

0.0 0.0
0.0 0.0
0.0 0.0


In [10]:
met_def_0_1_dist = Squared_distance_Ebin_field(tens_0_def, 
                                           torch.inverse(torch_tens_1), 
                                           1./2, 
                                           mask_union).detach().numpy()
G_def = torch.stack((tens_0_def,
                 torch.inverse(torch_tens_1)))
dim = 3.
kmean_def = get_karcher_mean(G_def, 1./dim)



NameError: name 'tens_0_def' is not defined

In [23]:
itkview(met_0_1_dist)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [24]:
itkview(met_def_0_1_dist)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [25]:
itkview(kmean[:,:,:,0,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [26]:
itkview(kmean_def[:,:,:,0,0])

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageD3; pro…

In [29]:
tens_0_def.device

device(type='cpu')

In [31]:
torch.from_numpy(tens_0).cuda().device

device(type='cuda', index=1)

In [32]:
torch.zeros((3,3)).device

device(type='cuda', index=1)

In [64]:
SQRT3 = 1.73205080756887729352744634151
def sym_eigenvals(g):
  # g.shape is [h,w,d,3,3]
  # Cardano algorithm from https://www.mpi-hd.mpg.de/personalhomes/globes/3x3/index.html
  # Determine coefficients of characteristic poynomial. We write
  #       | a   d   f  |
  #  A =  | d*  b   e  |
  #       | f*  e*  c  |
  de = g[:,:,:,0,1] * g[:,:,:,1,2]                                   # d * e
  dd = g[:,:,:,0,1] ** 2                                         # d^2
  ee = g[:,:,:,1,2] ** 2                                         # e^2
  ff = g[:,:,:,0,2] ** 2                                         # f^2
  m  =  g[:,:,:,0,0] +  g[:,:,:,1,1] +  g[:,:,:,2,2]
  c1 = g[:,:,:,0,0]*g[:,:,:,1,1] + g[:,:,:,0,0]*g[:,:,:,2,2] + g[:,:,:,1,1]*g[:,:,:,2,2] - (dd + ee + ff)        # a*b + a*c + b*c - d^2 - e^2 - f^2
  c0 = g[:,:,:,2,2]*dd + g[:,:,:,0,0]*ee + g[:,:,:,1,1]*ff -  g[:,:,:,0,0]*g[:,:,:,1,1]*g[:,:,:,2,2] - 2.0 * g[:,:,:,0,2]*de # c*d^2 + a*e^2 + b*f^2 - a*b*c - 2*f*d*e

  p = m * m - 3.0*c1
  q = m*(p - (3.0/2.0)*c1) - (27.0/2.0)*c0
  sqrt_p = torch.sqrt(torch.abs(p))

  phi = 27.0 * ( 0.25*c1*c1*(p - c1) + c0*(q + 27.0/4.0*c0))
  phi = (1.0/3.0) * torch.atan2(torch.sqrt(torch.abs(phi)), q)
  
  c = sqrt_p*torch.cos(phi)
  s = (1.0/SQRT3)*sqrt_p*torch.sin(phi)

  #w[1]  = (1.0/3.0)*(m - c)
  #w[2]  = w[1] + s
  #w[0]  = w[1] + c
  #w[1] -= s

  lambda2 = (1.0/3.0)*(m - c)
  lambda3 = lambda2 + s
  lambda1 = lambda2 + c
  
  eigenVals = torch.stack((lambda1, lambda2 - s, lambda3))
  return(eigenVals)

In [74]:
def eigen_direct(g, mask, get_eigenvectors=False):
  # g.shape is [h,w,d,3,3]
  # returned eigenvalue shape is [3,h,w,d]
  # returned eigenvector shape is [3,3,h,w,d]
  # This implementation based on
  # Hasan KM, Basser PJ, Parker DL, Alexander AL.
  # Analytical computation of the eigenvalues and eigenvectors in DT-MRI.
  # J Magn Reson 2001; 152: 41-47.

  # Precompute some commonly used values
  # D = spdMatrix ( to avoid pointer math)
  # where spdMatrix is a vector of the upper diagonal portion of the spd matrix 
  # following index map for reference since
  # D is always symmetric:
  #    | D00 D01 D02 |   | Dxx Dxy Dxz |   | 0 1 2 |
  # D = | D10 D11 D12 | = | Dyx Dyy Dyz | = | 1 3 4 |
  #    | D20 D21 D22 |   | Dzx Dzy Dzz |   | 2 4 5 |
  # DSq = spdMatrix .* spdMatrix
  # We don't need DSq0, DSq3, DSq5, so don't compute them
  D0 = g[:,:,:,0,0]
  D1 = g[:,:,:,0,1]
  D2 = g[:,:,:,0,2]
  D3 = g[:,:,:,1,1]
  D4 = g[:,:,:,1,2]
  D5 = g[:,:,:,2,2]
  DSq1 = D1 * D1
  DSq2 = D2 * D2
  DSq4 = D4 * D4


  # Compute the eigenvalues
  #
  # Compute the 3 principal invariants I1, I2, I3 of D
  # I1 = trace(D)
  # I2 = (DxxDyy + DxxDzz + DyyDzz) - ((Dxy)^2 + (Dxz)^2 + (Dyz)^2)
  # I3 = det(D) = DxxDyyDzz + 2DxyDxzDyz - (Dzz(Dxy)^2 + Dyy(Dxz)^2 +
  # Dxx(Dyz)^2)

  #RealTensorValueT I1, I2, I3, I1div3;
    
  I1 = D0 + D3 + D5
  I2 = D0*D3 + D0*D5 + D3*D5 - (DSq1 + DSq2 + DSq4)
  I3 = D0*D3*D5 + 2*D1*D2*D4 - (D5*DSq1 + D3*DSq2 + D0*DSq4)
  I1div3 = I1/3

  # Compute rotationally-invariant variables n and s
  # n = (I1/3)^2 - I2/3
  # s = (I1/3)^3 - I1*I2/6 + I3/2

  #RealTensorValueT n, sqrtn, s;
  n = I1div3 * I1div3 - I2/3.0
  s = I1div3 * I1div3 * I1div3 - I1*I2/6.0 + I3/2.0
  sqrtn = torch.sqrt(n)

  # Now check for some degenerate cases.
  # If these occur, default to the standard eigen analysis.
  # These basically enforce that the spdMatrix is in fact
  # symmetric positive definite.

  degen_8 = False
  if (torch.any(I3 < 1e-7)):
    print('degenerate case 1')
  if (torch.any(D0 < 0)):
    print('degenerate case 2')
  if (torch.any(D3 < 0)):
    print('degenerate case 3')
  if (torch.any(D5 < 0)):
    print('degenerate case 4')
  if (torch.any(D0 * D3 < DSq1)):
    print('degenerate case 5')
  if (torch.any(D0 * D5 < DSq2)):
    print('degenerate case 6')
  if (torch.any(D3 * D5 < DSq4)):
    print('degenerate case 7')
  if (torch.any(n < 1e-7)):
    degen_8 = True
    print('degenerate case 8')
    degen_8_idx = torch.nonzero(n < 1e-7, as_tuple=True)
    #print(idx[0][0],idx[1][0],idx[2][0])
    #print(g[idx[0][0],idx[1][0],idx[2][0]])
    #n[n < 1e-7] = 1e-7


  # Compute phi = (acos((s/n) * sqrt(1/n)) / 3)
  #RealTensorValueT phi;
  phi = torch.acos( (s/n) * 1/sqrtn) / 3.0

  # Now compute the eigenvalues
  # lambda1 = I1/3 + 2*sqrt(n)*cos(phi)
  # lambda2 = I1/3 - 2*sqrt(n)*cos(pi/3 + phi)
  # lambda3 = I1/3 - 2*sqrt(n)*cos(pi/3 - phi)
  # Due to trace invariance,
  # lambda3 also = I1 - lambda1 - lambda2

  #RealTensorValueT lambda1, lambda2, lambda3;
  lambda1 = I1div3 + 2 * sqrtn * torch.cos(phi)
  lambda2 = I1div3 - 2*sqrtn * torch.cos(torch.pi/3.0 + phi)
  lambda3 = I1 - lambda1 - lambda2

  if degen_8:
    lambda1[degen_8_idx] = 1
    lambda2[degen_8_idx] = 1
    lambda3[degen_8_idx] = 1
  
  eigenVals = torch.stack((lambda1, lambda2, lambda3))

  if get_eigenvectors:
    eigenVecs = torch.zeros((3,3,*g.shape[0:3]),device=g.device)
    # Compute the eigenvectors
    # This for loop could compute all 3 eigenvectors
    # instead of just the first two, but it is more
    # efficient to obtain the third eigenvector by
    # computing the cross product of the first two
    # eigenvectors.
    for ii in range(2):
      # Compute helper variables A, B, C
      # Ai = Dxx - eigenVals[i]
      # Bi = Dyy - eigenVals[i]
      # Ci = Dzz - eigenVals[i]
      #RealTensorValueT A, B, C;
      A = D0 - eigenVals[ii]
      B = D3 - eigenVals[ii]
      C = D5 - eigenVals[ii]

      # Compute eigenvec components x, y and z
      # eix = (DxyDyz - BiDxz)(DxzDyz - CiDxy)
      # eiy = (DxzDyz - CiDxy)(DxzDxy - AiDyz)
      # eiz = (DxyDyz - BiDxz)(DxzDxy - AiDyz)
      # recognizing common components, this becomes
      # eix = term1 * term2
      # eiy = term2 * term3
      # eiz = term1 * term3
      #RealTensorValueT term1, term2, term3;
      #RealTensorValueT ex, ey, ez;
      term1 = D1*D4 - B*D2
      term2 = D2*D4 - C*D1
      term3 = D2*D1 - A*D4
      ex = term1 * term2
      ey = term2 * term3
      ez = term1 * term3

      # Now normalize the vector
      # e = [ex ey ez]
      # eigenVec = e / sqrt(e'e)
      # RealTensorValueT norm, sqrtnorm;
      norm = ex * ex + ey * ey + ez * ez
      sqrtnorm = torch.sqrt(norm)
      eigenVecs[ii,0] = ex / sqrtnorm
      eigenVecs[ii,1] = ey / sqrtnorm
      eigenVecs[ii,2] = ez / sqrtnorm
      if degen_8:
        eigenVecs[0,0,degen_8_idx] = 1
        eigenVecs[0,1,degen_8_idx] = 0
        eigenVecs[0,2,degen_8_idx] = 0
        eigenVecs[1,0,degen_8_idx] = 0
        eigenVecs[1,1,degen_8_idx] = 1
        eigenVecs[1,2,degen_8_idx] = 0
        eigenVecs[2,0,degen_8_idx] = 0
        eigenVecs[2,1,degen_8_idx] = 0
        eigenVecs[2,2,degen_8_idx] = 1

    # Finally, compute the third eigenvector
    # e[2] = e[0] x e[1]
    # TODO better to switch to einsum here?
    eigenVecs[2,0] = eigenVecs[0,1] * eigenVecs[1,2] - eigenVecs[0,2] * eigenVecs[1,1]
    eigenVecs[2,1] = eigenVecs[0,2] * eigenVecs[1,0] - eigenVecs[0,0] * eigenVecs[1,2]
    eigenVecs[2,2] = eigenVecs[0,0] * eigenVecs[1,1] - eigenVecs[0,1] * eigenVecs[1,0]
  else:
    eigenVecs = None
  return(eigenVals, eigenVecs)


In [12]:
%%time
#a = torch.rand(5000, 3, 3,dtype=torch.double)
#a = 0.5 * (a + a.transpose(1, 2))
a = torch.rand(145,174,145, 3, 3,dtype=torch.double)
a = 0.5 * (a + a.transpose(3, 4))
w, wv = torch.symeig(a,True)  # fast (~0.0006s)


CPU times: user 14.7 s, sys: 720 ms, total: 15.4 s
Wall time: 15.6 s


torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)


In [13]:
%%time
t, tv = vlinalg.vSymEig(a.permute((1,2,0)).reshape((1,9,-1,1,1)),True)
e = t.reshape((3,-1)).permute((1,0))
ev = tv.reshape((3,3,-1)).permute((2,0,1))

RuntimeError: number of dims don't match in permute

In [14]:
wediff = torch.abs(w-e)
print(torch.min(w), torch.min(e))
print(torch.min(wediff), torch.max(wediff))
awv = torch.einsum('...ij,...jk->...ik',a,wv)
wwv = torch.einsum('...i,...ji->...ji',w,wv)
print(torch.max(torch.abs(awv-wwv)))
aev = torch.einsum('...ij,...jk->...ik',a,ev.double())
eev = torch.einsum('...i,...ji->...ji',e,ev)
print(torch.max(torch.abs(aev-eev)))


NameError: name 'e' is not defined

In [140]:
print(a.shape,wv.shape)

torch.Size([5000, 3, 3]) torch.Size([0])


In [123]:
%%time
se = sym_eigenvals(a.reshape((1,1,-1,3,3))).reshape((-1,3))

CPU times: user 5.71 ms, sys: 205 µs, total: 5.91 ms
Wall time: 4.31 ms


In [124]:
wsediff = torch.abs(w-se)
print(torch.min(w), torch.min(se))
print(torch.min(wsediff), torch.max(wsediff))
esediff = torch.abs(e-se)
print(torch.min(esediff), torch.max(esediff))


tensor(-1.0691, dtype=torch.float64) tensor(-1.0691, dtype=torch.float64)
tensor(0., dtype=torch.float64) tensor(3.1061, dtype=torch.float64)
tensor(2.3965e-08, dtype=torch.float64) tensor(3.1061, dtype=torch.float64)


In [199]:
%%time
t, tv = eigen_direct(a.reshape((1,1,-1,3,3)),None,True)
ed = t.reshape((-1,3))
ev = tv.reshape((-1,3,3))

degenerate case 1
degenerate case 5
degenerate case 6
degenerate case 7
CPU times: user 28.5 ms, sys: 4.61 ms, total: 33.1 ms
Wall time: 8.3 ms


In [200]:
weddiff = torch.abs(w-ed)
print(torch.min(w), torch.min(ed))
print(torch.min(weddiff), torch.max(weddiff))
edediff = torch.abs(ed-e)
print(torch.min(edediff), torch.max(edediff))
aev = torch.einsum('...ij,...kj->...ki',a,ev.double())
edev = torch.einsum('...i,...ij->...ji',ed,ev)
print(torch.max(torch.abs(aev-edev)))

tensor(-1.0677, dtype=torch.float64) tensor(-1.0677, dtype=torch.float64)
tensor(6.0333e-06, dtype=torch.float64) tensor(3.0615, dtype=torch.float64)
tensor(6.0341e-06, dtype=torch.float64) tensor(3.0615, dtype=torch.float64)
tensor(2.2842, dtype=torch.float64)


In [201]:
print(torch.matmul(a[0],ev[0,0,:].double()))
print(a[0].shape, ev[0].shape)
print(torch.einsum('...ij,...kj->...ik',a,ev.double())[0])
print(ev[0,0,:])
print(ed[0,1] * ev[0,0,:])
print(torch.einsum('...i,...ji->...j',ed,ev.double())[0])

tensor([1.2166, 1.3087, 0.9797], dtype=torch.float64)
torch.Size([3, 3]) torch.Size([3, 3])
tensor([[1.2166, 1.1558, 1.3276],
        [1.3087, 1.1808, 1.3656],
        [0.9797, 0.9417, 0.9597]], dtype=torch.float64)
tensor([0.6093, 0.7432, 0.4659])
tensor([1.0776, 1.3145, 0.8240])
tensor([3.1537, 2.9598, 3.3249], dtype=torch.float64)


In [204]:
aev = torch.matmul(a[0],ev[0,:,0].double())
print(aev / ev[0,:,0])
print(1.99*ev[0,0,:])
print(1.76*ev[0,0,:])
print(2.10*ev[0,0,:])

tensor([2.1648, 2.0386, 1.4436], dtype=torch.float64)
tensor([1.2125, 1.4790, 0.9272])
tensor([1.0723, 1.3081, 0.8200])
tensor([1.2795, 1.5608, 0.9784])


In [17]:
%%time
s3, s3v = Sym3Eig.apply(a.reshape((-1,3,3)))

CPU times: user 1.62 s, sys: 263 ms, total: 1.89 s
Wall time: 1.07 s


In [18]:
ws3diff = torch.abs(torch.sort(w)[0]-torch.sort(s3)[0])
print(torch.min(w), torch.min(s3))
print(torch.min(ws3diff), torch.max(ws3diff))
es3diff = torch.abs(e-s3)
print(torch.min(es3diff), torch.max(es3diff))
as3v = torch.einsum('...ij,...jk->...ik',a,s3v.double())
s3s3v = torch.einsum('...i,...ji->...ji',s3,s3v)
print(torch.max(torch.abs(as3v-s3s3v)))


RuntimeError: The size of tensor a (145) must match the size of tensor b (3658350) at non-singleton dimension 2

In [110]:
%%time
a = a.cuda()
w, _ = torch.symeig(a)  # slow (~0.9s)

CPU times: user 1.06 s, sys: 7.75 s, total: 8.81 s
Wall time: 8.82 s


In [111]:
%%time
e = vlinalg.vSymEig(a.permute((1,2,0)).reshape(1,9,-1,1,1))[0].reshape((3,-1)).permute((1,0))

CPU times: user 4.7 ms, sys: 106 µs, total: 4.81 ms
Wall time: 3.59 ms


In [112]:
wediff = torch.abs(w-e)
print(torch.min(wediff), torch.max(wediff))

tensor(4.4409e-14, device='cuda:0', dtype=torch.float64) tensor(1.1909e-07, device='cuda:0', dtype=torch.float64)


In [113]:
%%time
ed = eigen_direct(a.reshape((1,1,-1,3,3)),False)[0].reshape((-1,3))

degenerate case 1
degenerate case 5
degenerate case 6
degenerate case 7
CPU times: user 4.46 ms, sys: 0 ns, total: 4.46 ms
Wall time: 3.83 ms


In [116]:
%%time
s3, _ = Sym3Eig.apply(a)

CPU times: user 2.33 ms, sys: 288 µs, total: 2.61 ms
Wall time: 1.75 ms


In [117]:
ws3diff = torch.abs(w-s3)
print(torch.min(ws3diff), torch.max(ws3diff))
es3diff = torch.abs(e-s3)
print(torch.min(es3diff), torch.max(es3diff))

tensor(0., device='cuda:0', dtype=torch.float64) tensor(2.9406, device='cuda:0', dtype=torch.float64)
tensor(4.5131e-14, device='cuda:0', dtype=torch.float64) tensor(2.9406, device='cuda:0', dtype=torch.float64)


In [213]:
print(a.shape)

torch.Size([5000, 3, 3])
